In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 读取数据
data = pd.read_csv('data/train.csv')

# 数据预处理：提取磁通密度（1024个采样点），并进行标准化
X_raw = data.iloc[:, 4:]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

# 提取波形标签
y = data['励磁波形']

In [2]:
from scipy.fft import fft
import numpy as np

#	 
def extract_time_domain_features(X): 
    features = pd.DataFrame() 
    features['max'] = np.max(X, axis=1) 
    features['min'] = np.min(X, axis=1) 
    features['mean'] = np.mean(X, axis=1) 
    features['std'] = np.std(X, axis=1)
    features['peak_to_peak'] = np.max(X, axis=1) - np.min(X, axis=1) 
    return features

# 频域特征提取函数
def extract_frequency_domain_features(X): 
    features = pd.DataFrame()
    # 对每一行进行傅里叶变换并提取频域特征
    fft_feature = fft(X, axis=1)
    features['fft_max_freq'] = np.max(np.abs(fft_feature), axis=1)
    features['fft_energy'] = np.sum(np.abs(fft_feature)**2, axis=1)
    return features

# 提取特征
time_features = extract_time_domain_features(X_scaled)
freq_features = extract_time_domain_features(X_scaled)

# 合并所有特征
X_features = pd.concat([time_features, freq_features], axis=1)


In [3]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=10)
X_selected = selector.fit_transform(X_features, y)

selected_features = X_features.columns[selector.get_support()]
print(f"Selected features: {selected_features}")


Selected features: Index(['max', 'min', 'mean', 'std', 'peak_to_peak', 'max', 'min', 'mean',
       'std', 'peak_to_peak'],
      dtype='object')


In [4]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from deap import base, creator, tools, algorithms

selector = SelectKBest(f_classif, k=10)
X_selected = selector.fit_transform(X_features, y)

selected_features = X_features.columns[selector.get_support()]
print(f"Selected features: {selected_features}")

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 0.1, 10.0)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalSVM(individual):
    C, gamma = individual
    model = SVC(C=C, gamma=gamma)
    score = cross_val_score(model, X_train, y_train, cv=5).mean()
    return score,


toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate",evalSVM)


population = toolbox.population(n=20)
NGEN = 10
CXPB = 0.7
MUTPB = 0.2


result, logbook = algorithms.eaSimple(population, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN,
verbose=True, stats=None)

best_individual = tools.selBest(result, 1)[0]
print(f"Best C: {best_individual[0]}, Best Gamma:{best_individual[1]}")


best_model = SVC(C=best_individual[0], gamma=best_individual[1])
best_model.fit(X_train, y_train)

Selected features: Index(['max', 'min', 'mean', 'std', 'peak_to_peak', 'max', 'min', 'mean',
       'std', 'peak_to_peak'],
      dtype='object')
gen	nevals
0  	20    


TypeError: mutGaussian() missing 1 required positional argument: 'sigma'